In [5]:
import pandas as pd
import httpx
import asyncio
import ujson as json
from datetime import datetime
import os
from typing import Dict, TypeAlias

from script import FedInvestFetcher
JSON: TypeAlias = dict[str, "JSON"] | list["JSON"] | str | int | float | bool | None

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
def runner(dates):
    async def build_tasks(client: httpx.AsyncClient, dates):
        tasks = await FedInvestFetcher(
            use_ust_issue_date=True, error_verbose=True
        )._build_fetch_tasks_historical_cusip_prices(
            client=client, dates=dates, max_concurrent_tasks=5
        )
        return await asyncio.gather(*tasks)

    async def run_fetch_all(dates):
        limits = httpx.Limits(max_connections=5)
        async with httpx.AsyncClient(limits=limits) as client:
            all_data = await build_tasks(client=client, dates=dates)
            return all_data

    results = asyncio.run(run_fetch_all(dates=dates))
    return dict(results)

In [8]:
def get_business_days_groups(start_date: datetime, end_date: datetime, group_size=3):
    date_range = pd.date_range(start=start_date, end=end_date, freq="B")
    business_day_groups = [
        [bday.to_pydatetime() for bday in date_range[i : i + group_size].tolist()]
        for i in range(0, len(date_range), group_size)
    ]
    return business_day_groups


start_date = datetime(2024, 8, 12)
end_date = datetime(2024, 8, 16)
weeks = get_business_days_groups(start_date, end_date, group_size=5)

In [9]:
for week in weeks:
    dict_df: Dict[datetime, pd.DataFrame] = runner(dates=week)
    output_directory = r"C:\Users\chris\CUSIP-Set"
    for key, df in dict_df.items():
        records = df.to_dict(orient='records')
        json_structure = {"data": records}
        file_name = f"{key.strftime("%Y-%m-%d")}.json"
        file_path = os.path.join(output_directory, file_name)
        with open(file_path, 'w') as json_file:
            json.dump(json_structure, json_file, indent=4) 

2024-08-18 10:37:56,054 - script - DEBUG - UST Prices - 2024-08-12 00:00:00 Payload: {'priceDate.month': 8, 'priceDate.day': 12, 'priceDate.year': 2024, 'submit': 'Show Prices'}
2024-08-18 10:37:56,078 - script - DEBUG - UST Prices - 2024-08-13 00:00:00 Payload: {'priceDate.month': 8, 'priceDate.day': 13, 'priceDate.year': 2024, 'submit': 'Show Prices'}
2024-08-18 10:37:56,079 - script - DEBUG - UST Prices - 2024-08-14 00:00:00 Payload: {'priceDate.month': 8, 'priceDate.day': 14, 'priceDate.year': 2024, 'submit': 'Show Prices'}
2024-08-18 10:37:56,080 - script - DEBUG - UST Prices - 2024-08-15 00:00:00 Payload: {'priceDate.month': 8, 'priceDate.day': 15, 'priceDate.year': 2024, 'submit': 'Show Prices'}
2024-08-18 10:37:56,081 - script - DEBUG - UST Prices - 2024-08-16 00:00:00 Payload: {'priceDate.month': 8, 'priceDate.day': 16, 'priceDate.year': 2024, 'submit': 'Show Prices'}
2024-08-18 10:37:56,425 - script - DEBUG - UST Prices - 2024-08-15 00:00:00 Redirecting to https://savingsbond